Берем результаты других ноутбуков и объединяем их тут

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ohe_noise.tabddpm_ohe_noise import *

from tqdm.notebook import tqdm

from os import listdir
from os.path import isfile, join, isdir

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

In [3]:
datename = 'shoppers'

In [4]:
path = f'./eval/total/'
dirs = [join(path, f) for f in listdir(path) if isdir(join(path, f))]

In [5]:
results = []
for d in dirs:
    if datename in d:
        res = [pd.read_csv(join(d, f), header=[0, 1], skipinitialspace=True) for f in listdir(d) if isfile(join(d, f)) and 'csv' in f]
        results.extend(res)

results = pd.concat(results)
results = results.rename(columns={'Unnamed: 0_level_0':'', 
                                  'Unnamed: 1_level_0':'', 
                                  'Unnamed: 2_level_0':''})

In [9]:
import numpy as np
import json

with open('/workspace/TabDDPM_copy/TabDDPM/Model_names_simple.json', 'r') as openfile:
    naming = json.load(openfile)

def get_positions(s, substring):
    positions = []
    start = 0
    while True:
        start = s.find(substring, start)
        if start == -1:
            break
        positions.append(start)
        start += len(substring)
    return positions
    
def get_simple_name(model_name):
    try:
        pos = get_positions(model_name, '_')[-2]
        model_name_short = model_name[pos+1:]

        if naming.get(model_name_short):
            return naming[model_name_short]
        else:
            pos = get_positions(model_name, '_')[-3]
            model_name_short = model_name[pos+1:]
            return naming[model_name_short]
    except:
        return 'TabDDPM'

results[('', 'Model name')] = results[('', 'Type')].apply(lambda x: get_simple_name(x))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = results[[
    ('', 'Model name') ,
    ('similarity', 'Column Shapes Score, %') ,
    ('similarity', 'Column Pair Trends Score, %') ,
    ('similarity', 'Overall Score (Average), %') ,
    ('similarity', 'Error rate (%) of column-wise density estimation, %') ,
    ('similarity', 'Error rate (%) of column-wise density estimation std, %') ,
    ('similarity', 'Error rate (%) of pair-wise column correlation score, %') ,
    ('similarity', 'Error rate (%) of pair-wise column correlation score std, %') ,
    ('mle', 'ROC - AUC обучения на синтетических данных') ,
    ('mle', 'ROC - AUC обучения на синтетических данных, std') ,
    ('detection', 'Score') ,
    ('DCR', 'Score') ,
    ('quality', 'alpha precision') ,
    ('quality', 'beta recall') ,
]].sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

df

similarity                              \
    Model name Column Shapes Score, % Column Pair Trends Score, %   
0   Constant 2              98.453035                   97.855622   
1   Constant 1              98.395963                   97.749331   
2   Constant 4              98.423498                   97.715089   
3      Steps 4              98.403973                   97.660239   
4   Constant 3              98.577193                   97.464313   
5      Steps 5              98.276311                   97.755681   
6     Cosine 2              98.200715                   97.715414   
7      Steps 3              98.388453                   97.403744   
8     Cosine 3              98.112102                   97.531120   
9     Cosine 1              98.395462                   97.018327   
10    Linear 2              98.029497                   97.339668   
11     Steps 2              98.353409                   96.997970   
12    Linear 1              98.307851                   96.977806   
13  Constant 5              97.814224                   97.432189   
14     Steps 1              98.216735                   95.334466   
15  Constant 6              96.427463                   96.663761   
16     TabDDPM              97.411713                   92.929939   

                               \
   Overall Score (Average), %   
0                   98.154329   
1                   98.072647   
2                   98.069294   
3                   98.032106   
4                   98.020753   
5                   98.015996   
6                   97.958064   
7                   97.896099   
8                   97.821611   
9                   97.706894   
10                  97.684582   
11                  97.675689   
12                  97.642829   
13                  97.623206   
14                  96.775601   
15                  96.545612   
16                  95.170826   

                                                        \
   Error rate (%) of column-wise density estimation, %   
0                                             1.546965   
1                                             1.604037   
2                                             1.576502   
3                                             1.596027   
4                                             1.422807   
5                                             1.723689   
6                                             1.799285   
7                                             1.611547   
8                                             1.887898   
9                                             1.604538   
10                                            1.970503   
11                                            1.646591   
12                                            1.692149   
13                                            2.185776   
14                                            1.783265   
15                                            3.572537   
16                                            2.588287   

                                                            \
   Error rate (%) of column-wise density estimation std, %   
0                                                 0.799892   
1                                                 0.877375   
2                                                 0.786842   
3                                                 0.704324   
4                                                 0.573201   
5                                                 0.827217   
6                                                 0.859476   
7                                                 0.761531   
8                                                 0.885505   
9                                                 0.826794   
10                                                1.017441   
11                                                0.796192   
12                                                0.800809   
13                                                1.04

In [12]:
df[df[('', 'Model name')].isin(['TabDDPM', 'Constant 2', 'Constant 4', 'Cosine 3'])]

similarity                              \
    Model name Column Shapes Score, % Column Pair Trends Score, %   
0   Constant 2              98.453035                   97.855622   
2   Constant 4              98.423498                   97.715089   
8     Cosine 3              98.112102                   97.531120   
16     TabDDPM              97.411713                   92.929939   

                               \
   Overall Score (Average), %   
0                   98.154329   
2                   98.069294   
8                   97.821611   
16                  95.170826   

                                                        \
   Error rate (%) of column-wise density estimation, %   
0                                             1.546965   
2                                             1.576502   
8                                             1.887898   
16                                            2.588287   

                                                            \
   Error rate (%) of column-wise density estimation std, %   
0                                                 0.799892   
2                                                 0.786842   
8                                                 0.885505   
16                                                1.478745   

                                                            \
   Error rate (%) of pair-wise column correlation score, %   
0                                                 2.144378   
2                                                 2.284911   
8                                                 2.468880   
16                                                7.070061   

                                                                \
   Error rate (%) of pair-wise column correlation score std, %   
0                                                     1.284139   
2                                                     1.368165   
8                                                     1.456847   
16                                                    6.710063   

                                          mle  \
   ROC - AUC обучения на синтетических данных   
0                                    0.914732   
2                                    0.936201   
8                                    0.928131   
16                                   0.930595   

                                                   detection       DCR  \
   ROC - AUC обучения на синтетических данных, std     Score     Score   
0                                         0.006875  0.947366  0.510769   
2                                         0.005396  0.938000  0.509777   
8                                         0.003714  0.922970  0.514283   
16                                        0.003077  0.872748  0.524466   

           quality              
   alpha precision beta recall  
0         0.955273    0.526509  
2         0.962697    0.526929  
8         0.946370    0.535634  
16        0.918250    0.537797

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

\
              Model                              Type      Data   
0   tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_001  shoppers   
1   tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_const_0  shoppers   
2   tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_1  shoppers   
3   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_3  shoppers   
4   tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_01  shoppers   
5   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_4  shoppers   
6   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_8  shoppers   
7   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_2  shoppers   
8   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_9  shoppers   
9   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_7  shoppers   
10  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_6  shoppers   
11  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_1  shoppers   
12  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_5  shoppers   
13  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_25  shoppers   
14  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_0  shoppers   
15  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_5  shoppers   
16          TabDDPM                           tabddpm  shoppers   

        base_metrics                                                  \
   Original Logistic Synthetic Logistic Original Tree Synthetic Tree   
0           0.674157           0.686957      0.637363       0.581081   
1           0.674157           0.691630      0.629712       0.606987   
2           0.674157           0.676790      0.637363       0.602679   
3           0.674157           0.685590      0.637363       0.577982   
4           0.674157           0.682713      0.637363       0.627358   
5           0.674157           0.670968      0.637363       0.577878   
6           0.674157           0.679654      0.637363       0.604119   
7           0.674157           0.679739      0.637363       0.619385   
8           0.674157           0.689038      0.637363       0.563991   
9           0.674157           0.676923      0.637363       0.615034   
10          0.674157           0.677060      0.637363       0.558685   
11          0.674157           0.696833      0.637363       0.596882   
12          0.674157           0.669546      0.637363       0.607710   
13          0.674157           0.659389      0.637363       0.613391   
14          0.674157           0.669584      0.637363       0.559091   
15          0.674157           0.673913      0.637363       0.603239   
16          0.680272           0.620278      0.635347       0.589569   

                                                               similarity  \
   Accuracy Loss Logistic, % Accuracy Loss Tree, % Column Shapes Score, %   
0                  -1.898551              8.830382              98.453035   
1                  -2.591777              3.608771              98.395963   
2                  -0.390456              5.441810              98.423498   
3                  -1.695779              9.316672              98.403973   
4                  -1.269147              1.569616              98.577193   
5                   0.473118              9.332918              98.276311   
6                  -0.815296              5.215813              98.200715   
7                  -0.827887              2.820576              98.388453   
8                  -2.207308             11.511706              98.112102   
9                  -0.410256              3.503260              98.395462   
10                 -0.430586             12.344180              98.029497   
11                 -3.363499              6.351279              98.353409   
12                  0.683945              4.652436              98.307851   
13                  2.190684              3.761078              97.814224   
14                  0.678337             12.280564              98.216735   
15                  0.036232              5.353902              96.427463   
16                  8.819085        

In [8]:
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False
                   ).reset_index(drop=True).to_excel(f'./_{datename}_notebooks/Result_united.xlsx')